<center>

**Vježba 1 - Priprema i instalacija Apache PySparka**

</center>

U ovoj vježbi želimo vidjeti nekoliko postupaka čišćenja. Koncentrirati ćemo se na ETL nad bazama podataka. Prvo instaliramo sve python module koji će nam za to trebati. Posebno se koncentriramo na module *pandas*, *sqlalchemy* i *pyspark*.

In [ ]:
%%bash
pip install -U pip
pip install numpy pandas sqlalchemy matplotlib seaborn scikit-learn pyspark

Privremeno stvorimo direktorij u koji ćemo staviti Dockerfile za pripremu Apache PySpark image-a.

In [ ]:
%%bash
mkdir ./spark_tmp

Generiramo Dockerfile. Krenemo od image-a koji ima osnovni python 3.10.6.
Prvo u image instaliramo pakete *curl* i *wget* koje ćemo koristiti za kreiranje image-a.
Za python instaliamo module *pandas* (PySpark može raditi s pandas-om, ali to nije distribuirano), te module za jupyter kernel.

Instaliramo OpenJDK 11 u /home/jdk-11.0.2.
Zatim instaliramo Apache Spark 3.3.2 u /home/spark.
Na kraju u /home/spark/jars dodamo JDBC driver za PostgreSQL bazu podataka. Ova lokacija je u Java path-u za Apache Spark.

Podesimo sigurnost za /tmp, te varijable za jupyter kernel. Na ovaj ćemo se kernel u docker image-u spojiti iz jupyter notebooka.

In [ ]:
%%writefile ./spark_tmp/Dockerfile
FROM python:3.10.6

RUN apt-get install curl wget
RUN pip install pandas ipython ipykernel

ENV SPARK_VERSION=3.3.2 \
HADOOP_VERSION=3 \
JAVA_VERSION=11

ENV JAVA_HOME="/home/jdk-${JAVA_VERSION}.0.2"
ENV PATH="${JAVA_HOME}/bin/:${PATH}"

RUN DOWNLOAD_URL="https://download.java.net/java/GA/jdk${JAVA_VERSION}/9/GPL/openjdk-${JAVA_VERSION}.0.2_linux-x64_bin.tar.gz" \
    && TMP_DIR="$(mktemp -d)" \
    && curl -fL "${DOWNLOAD_URL}" --output "${TMP_DIR}/openjdk-${JAVA_VERSION}.0.2_linux-x64_bin.tar.gz" \
    && mkdir -p "${JAVA_HOME}" \
    && tar xzf "${TMP_DIR}/openjdk-${JAVA_VERSION}.0.2_linux-x64_bin.tar.gz" -C "${JAVA_HOME}" --strip-components=1 \
    && rm -rf "${TMP_DIR}" \
    && java --version

RUN DOWNLOAD_URL_SPARK="https://dlcdn.apache.org/spark/spark-${SPARK_VERSION}/spark-${SPARK_VERSION}-bin-hadoop${HADOOP_VERSION}.tgz" \
    && wget --no-verbose -O apache-spark.tgz "${DOWNLOAD_URL_SPARK}"\
    && mkdir -p /home/spark \
    && tar -xf apache-spark.tgz -C /home/spark --strip-components=1 \
    && rm apache-spark.tgz

RUN DOWNLOAD_URL_JDBC="https://jdbc.postgresql.org/download/postgresql-42.6.0.jar" \
    && wget --no-verbose -P /home/spark/jars "${DOWNLOAD_URL_JDBC}"

ENV SPARK_HOME="/home/spark"
ENV PATH="${SPARK_HOME}/bin/:${PATH}"

ENV PYTHONPATH="${SPARK_HOME}/python/:$PYTHONPATH"
ENV PYTHONPATH="${SPARK_HOME}/python/lib/py4j-0.10.9.5-src.zip:$PYTHONPATH"

RUN chmod 777 /tmp
ENV JUPYTER_RUNTIME_DIR="/tmp"
ENV JUPYTER_ALLOW_INSECURE_WRITES=true

USER $NB_UID

Napravimo build docker image-a spark_aris:0.5
0.5 je verzija image-a - može biti i neka druga verzija.

Nakon builda možemo provjeriti da li se novi image vidi... provjerite docker nadogradnju za vscode.

In [ ]:
%%bash
cd ./spark_tmp
docker build -t spark_aris:0.5 ./

U našem virtualnom okruženju za ARIS laboratorij dodamo custom jupyter kernel koji se nalazi u docker image-u.
Prilikom pokretanja kernela, pokrećemo i docker image.

In [ ]:
%%bash
mkdir ../venv/share/jupyter/kernels/spark_aris

Primijetimo datoteku za specifikaciju komunikacije između jupyter notebooka i kernela (*connection-spec*), koju dodajemo u image na mjesto /tmp/connection-spec. Zatim se u image-u pokreće kernel s tim specifikacijama. Taj kernel otvara nekoliko portova kroz koje jupyter notebook komunicira sa kernelom.

Na taj se način pristupa python API-u Apache Sparka koji smo instalirali u /home/spark/python

In [ ]:
%%writefile ../venv/share/jupyter/kernels/spark_aris/kernel.json
{
 "argv": [
  "/usr/bin/docker",
  "run",
  "--network=host",
  "-v",
  "{connection_file}:/tmp/connection-spec",
  "spark_aris:0.5",
  "python",
  "-m",
  "ipykernel_launcher",
  "-f",
  "/tmp/connection-spec"
 ],
 "display_name": "spark_aris",
 "language": "python"
}

In [ ]:
%%writefile ../venv/share/jupyter/kernels/spark_aris/connection-spec
{
  "kernel_name": "",
  "hb_port": 51115,
  "ip": "0.0.0.0",
  "control_port": 51114,
  "stdin_port": 51113,
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "key": "19749810-8febfa748186a01da2f7b28c",
  "iopub_port": 51112,
  "shell_port": 51111
}

Sad možemo dodati jupyter notebook u kojem se spajanjem na *spark_aris* kernel dešava sljedeće:
- pokreće se Apache Spark docker image
- jupyter notebook se spaja na kernel u docker image-u
- kod iz jupyter notebooka može koristiti Apache PySpark